# Labeling Functions


In [1]:
import re
import glob
import emoji
import fasttext
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from nltk.tokenize import word_tokenize
from nltk import ngrams
from nltk.corpus import stopwords
from sentence_transformers import util

from snorkel.preprocess import preprocessor
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel
from sklearn.metrics import classification_report

In [ ]:
# defining data paths
prop_data_annotated = "../data/processed/propaganda_annotated.pkl"
gen_data_annotated = "../data/processed/genuine_annotated.pkl"
wanlp_prop_data_path = "../data/raw/task1_train.json"
weakly_labeled_data_path = "../data/processed/train.json"

# we need the lf_dev data to report the performance of LFs
# and then remove it with the test data from the unlabaled data
lf_dev_data_path = "../data/processed/lf_dev.json"
test_data_path = "../data/processed/test_gold.json"

# defining lexicons paths
proppy_lex_paths = "../data/raw/proppy_lexicons/*"
loaded_lex_path = "../data/raw/loaded-language-lexicons.csv"

# loading pre-trained encoders
fasttext_model_path = "../models/cc.ar.300.bin"

In [3]:
# defining the needed fields by the working labeling functions
# in case of doing the analysis from scratch, comment the variable

fields = [
    "tweetid",
    "user_profile_description",
    "tweet_text",
    "is_retweet",
    "quote_count",
    "reply_count",
    "like_count",
    "retweet_count",
    "hashtags",
    "urls",
    "user_mentions",
    "text",
    "emojis",
    "word_count",
    "docs",
    "is_irony",
    "is_hate",
    "xlmroberta_label",
]

In [4]:
df1 = pd.read_pickle(prop_data_annotated)[fields]
df2 = pd.read_pickle(gen_data_annotated)[fields]

In [ ]:
# the following lines handle date conversion

df1["account_creation_date"] = pd.to_datetime(df1.account_creation_date, unit="ms")
df2["account_creation_date"] = pd.to_datetime(df2.account_creation_date, unit="ns")

In [5]:
# combining the propaganda and genuine users data

unlabeled_data = pd.concat([df1, df2], ignore_index=True)
unlabeled_data = unlabeled_data.sample(frac=1.0).reset_index(drop=True)

In [6]:
unlabeled_data.head(2)

,tweetid,user_profile_description,tweet_text,is_retweet,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,text,emojis,word_count,docs,is_irony,is_hate,xlmroberta_label
0,1135592977227812865,الأخبـار العاجلة #عاجل مـن مـوقـع جريدة الرياض...,#عاجل:\nالمحكمة العليا: ثبوت رؤية هلال شوال.. ...,False,10.0,16.0,110,169.0,2,1,0,عاجل: المحكمة العليا: ثبوت رؤية هلال شوال.. وغ...,0,17,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.95331501960...","{'label': 'not offensive', 'score': 0.99942123...",{'sequence': 'عاجل: المحكمة العليا: ثبوت رؤية ...
1,1417191564229623808.0,الحساب الرسمي لـ إمارة منطقة القصيم - إنستقرام...,حمد سموه المولى عز وجل على ماتحظى به بلادنا من...,False,0.0,3.0,30.0,39.0,4,1,0,حمد سموه المولى عز وجل على ماتحظى به بلادنا من...,0,35,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.94064635038...","{'label': 'not offensive', 'score': 0.99974423...",{'sequence': 'حمد سموه المولى عز وجل على ماتحظ...
2,1263767921010057216.0,الحساب الرسمي للأمن العام السعودي,شرطة القصيم : ضبط مواطنٍ ومقيم استغلّا تصريح ا...,False,3.0,14.0,157.0,119.0,0,1,0,شرطة القصيم : ضبط مواطن ومقيم استغلا تصريح الس...,0,19,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.85628890991...","{'label': 'not offensive', 'score': 0.99958091...",{'sequence': 'شرطة القصيم : ضبط مواطن ومقيم اس...
3,1460686542133997573,None,«نزاهة»: ثبوت تورط (20) مواطناً ومقيماً بتعديل...,False,0.0,3.0,5,2.0,1,1,0,«نزاهة»: ثبوت تورط (20) مواطنا ومقيما بتعديل ح...,0,40,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.74483102560...","{'label': 'not offensive', 'score': 0.99972814...",{'sequence': '«نزاهة»: ثبوت تورط (20) مواطنا و...
4,870571946106400768,‏‏‎‎#الحرية هي أن تؤمن بها لك ولغيرك\nلكن تفرض...,RT @hzen2080: ساعات رومانس\nساعه اوديمار أوتوم...,True,0.0,0.0,0,0.0,0,0,1,RT : ساعات رومانس ساعه اوديمار أوتوماتيك 650﷼ ...,0,22,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.96675294637...","{'label': 'not offensive', 'score': 0.99922728...",{'sequence': 'RT : ساعات رومانس ساعه اوديمار أ...


In [7]:
unlabeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196612 entries, 0 to 196611
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweetid                   196610 non-null  object 
 1   user_profile_description  183084 non-null  object 
 2   tweet_text                196612 non-null  object 
 3   is_retweet                196612 non-null  bool   
 4   quote_count               196607 non-null  float64
 5   reply_count               196608 non-null  float64
 6   like_count                196608 non-null  object 
 7   retweet_count             196608 non-null  float64
 8   hashtags                  196612 non-null  int64  
 9   urls                      196612 non-null  int64  
 10  user_mentions             196612 non-null  int64  
 11  text                      196612 non-null  object 
 12  emojis                    196612 non-null  int64  
 13  word_count                196612 non-null  i

In [8]:
labeled = pd.read_json(lf_dev_data_path)
labeled.head(2)

,tweetid,tweet_text,text,tech,label
0,924924839902793728,RT @Amal_onzi: 🕊💕هُو جنْةبعِيني.,RT : هو جنةبعيني.,None,0
1,1074734231887187968,ر٣ #تركيا_تجاهر_بالمعاصي,ر٣ تركيا تجاهر بالمعاصي,smears - name-calling - loaded language,1
2,1089924333378682880,RT @al_raqi_8: 💝 طقم نسائي حصيره شكل ديور مغ...,RT : طقم نسائي حصيره شكل ديور مغناطيس ١ *ساعه ...,None,0
3,752398332270563328,RT @vvvv1l: اصحو ي ناس 👊\n\n #الثوره_الخمينيه_...,RT : اصحو ي ناس الثوره الخمينيه تتمزق,loaded language,1
4,1079749279089078272,RT @rood516: اطقم نسائيه من ماركة رولكس😍✨1\nسا...,RT : اطقم نسائيه من ماركة رولكس1 ساعه نسائيه م...,None,0


In [9]:
labeled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetid     500 non-null    int64 
 1   tweet_text  500 non-null    object
 2   text        500 non-null    object
 3   tech        48 non-null     object
 4   label       500 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 23.4+ KB


In [ ]:
test = pd.read_json(test_data_path)
test.head(2)

In [ ]:
test.info()

In [11]:
labeled_data = unlabeled_data[unlabeled_data.text.isin(labeled.text)]
labeled_data.head(2)

,tweetid,user_profile_description,tweet_text,is_retweet,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,text,emojis,word_count,docs,is_irony,is_hate,xlmroberta_label
318,841000033365446656,دام ابوي حولي ما علي خلاف.,يااخي اول شي اجابته صح ثاني شي في اجابه خطا لع...,False,0.0,0.0,0,0.0,1,0,3,يااخي اول شي اجابته صح ثاني شي في اجابه خطا لع...,0,19,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.53657037019...","{'label': 'not offensive', 'score': 0.99910682...",{'sequence': 'يااخي اول شي اجابته صح ثاني شي ف...
373,862407267840393216,شوي من هنا وشويتين من هناك...سياسة ومجتمع وشعر...,الا غصب تشتري 😂😂 https://t.co/cbVH13bh6J,False,0.0,0.0,0,0.0,0,0,0,الا غصب تشتري,2,3,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.68531942367...","{'label': 'not offensive', 'score': 0.99869710...","{'sequence': 'الا غصب تشتري', 'labels': ['tran..."
390,683277968156798976,‏‏‏(رب اغفر لي ولوالدي ولمن دخل بيتي مؤمناً ول...,RT @aldmgane: لم أشجع الهلال باحثاً عن رضاكم،\...,True,0.0,0.0,0,0.0,0,0,1,RT : لم أشجع الهلال باحثا عن رضاكم، ولم أمتدحه...,0,24,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.92064410448...","{'label': 'not offensive', 'score': 0.99944049...",{'sequence': 'RT : لم أشجع الهلال باحثا عن رضا...
652,1026259146000228352,‏‏‎‎#الحرية هي أن تؤمن بها لك ولغيرك\nلكن تفرض...,RT @ksavi1p: أول مرة أشوف هذا الفديو من هذه ال...,True,0.0,0.0,0,0.0,0,0,1,RT : أول مرة أشوف هذا الفديو من هذه الزاوية,7,10,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.75664991140...","{'label': 'not offensive', 'score': 0.99878746...",{'sequence': 'RT : أول مرة أشوف هذا الفديو من ...
1467,1069588123846356992,من قال اني لا ابوح ؟! قد أخبرت ربي كل شيء !!,#استقبال_مليوني\n اللهم اجعلنا اوفر عبادك حظا ...,False,0.0,0.0,0,0.0,1,0,0,استقبال مليوني اللهم اجعلنا اوفر عبادك حظا في ...,0,11,"[\n [\n {\n ""id"": 1,\n ""text"": ""...","{'label': 'nonsarcasm', 'score': 0.95686054229...","{'label': 'not offensive', 'score': 0.99981170...",{'sequence': 'استقبال مليوني اللهم اجعلنا اوفر...


In [12]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 318 to 196598
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tweetid                   500 non-null    object 
 1   user_profile_description  462 non-null    object 
 2   tweet_text                500 non-null    object 
 3   is_retweet                500 non-null    bool   
 4   quote_count               500 non-null    float64
 5   reply_count               500 non-null    float64
 6   like_count                500 non-null    object 
 7   retweet_count             500 non-null    float64
 8   hashtags                  500 non-null    int64  
 9   urls                      500 non-null    int64  
 10  user_mentions             500 non-null    int64  
 11  text                      500 non-null    object 
 12  emojis                    500 non-null    int64  
 13  word_count                500 non-null    int64  
 14  docs      

In [13]:
# removing the labeled tweets from the unlabeled data

unlabeled_data = unlabeled_data[~unlabeled_data.text.isin(labeled_data.text)]
unlabeled_data = unlabeled_data[~unlabeled_data.text.isin(test.text)]
unlabeled_data = unlabeled_data.reset_index(drop=True)

In [ ]:
labeled_data = labeled_data.reset_index(drop=True)
labels = [labeled[labeled.text == i].label.values[0] for i in labeled_data.text]
labeled_data["label"] = labels

In [14]:
# making sure the dataset doesn't contain any null values

subset = ["tweetid", "text", "quote_count", "xlmroberta_label"]
unlabeled_data = unlabeled_data.dropna(subset=subset)
unlabeled_data = unlabeled_data.reset_index(drop=True)

In [15]:
unlabeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196106 entries, 0 to 196105
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   tweetid                   196106 non-null  object 
 1   user_profile_description  182621 non-null  object 
 2   tweet_text                196106 non-null  object 
 3   is_retweet                196106 non-null  bool   
 4   quote_count               196106 non-null  float64
 5   reply_count               196106 non-null  float64
 6   like_count                196106 non-null  object 
 7   retweet_count             196106 non-null  float64
 8   hashtags                  196106 non-null  int64  
 9   urls                      196106 non-null  int64  
 10  user_mentions             196106 non-null  int64  
 11  text                      196106 non-null  object 
 12  emojis                    196106 non-null  int64  
 13  word_count                196106 non-null  i

In [16]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   tweetid                   500 non-null    object 
 1   user_profile_description  462 non-null    object 
 2   tweet_text                500 non-null    object 
 3   is_retweet                500 non-null    bool   
 4   quote_count               500 non-null    float64
 5   reply_count               500 non-null    float64
 6   like_count                500 non-null    object 
 7   retweet_count             500 non-null    float64
 8   hashtags                  500 non-null    int64  
 9   urls                      500 non-null    int64  
 10  user_mentions             500 non-null    int64  
 11  text                      500 non-null    object 
 12  emojis                    500 non-null    int64  
 13  word_count                500 non-null    int64  
 14  docs      

### User LFs


In [18]:
prop = 1
gen = 0
ab = -1

In [19]:
@labeling_function()
def missing_bio(example):
    """Label all tweets of a user as propaganda if they don't have a bio."""
    if pd.isna(example.user_profile_description):
        return prop
    else:
        return ab

In [21]:
@labeling_function()
def missing_loc(example):
    """Label all tweets of a user as propaganda if they don't have a location."""
    if pd.isna(example.user_reported_location):
        return prop
    else:
        return ab

In [22]:
@labeling_function()
def created_2018_2019(example):
    """Label all tweets of a user as propaganda if account is created 2018 or 2019."""
    if example.account_creation_date.year in [2018, 2019]:
        return prop
    else:
        return ab

In [23]:
@labeling_function()
def created_2011_2012(example):
    """Label all tweets of a user as transparent if account is created 2011 or 2012."""
    if example.account_creation_date.year in [2011, 2012]:
        return gen
    else:
        return ab

In [26]:
@preprocessor(memoize=False)
def tokenize_bio(example):
    """Tokenize text in the bio."""
    if not pd.isna(example.user_profile_description):
        example.bio_tokens = word_tokenize(example.user_profile_description)
    else:
        example.bio_tokens = None
    return example

In [27]:
@labeling_function(pre=[tokenize_bio])
def bio_keywords(example):
    """Label all tweets of a user as transparent if bio contains certain lexicons."""
    keys = ["الحساب", "الرسمي", "عضو", "رئيس", "كاتب", "إدارة"]
    if example.bio_tokens is not None:
        if any(np.in1d(keys, example.bio_tokens)):
            return gen
        else:
            return ab
    else:
        return ab

### Tweet LFs


In [29]:
@labeling_function()
def contain_url(example):
    """Label tweet as transparent if it contains a URL."""
    if example.urls > 0:
        return gen
    else:
        return ab

In [30]:
@labeling_function()
def contain_mention(example):
    """Label tweet as propaganda if it contains a mention."""
    if example.user_mentions > 0:
        return prop
    else:
        return ab

In [31]:
@labeling_function()
def labeling_sarcasm(example):
    """Label tweet as propaganda if it contains the name calling tech (sarcasm)."""
    if len(example.docs.entities) and example.sarcasm["label"] == "Sarcasm":
        return prop
    else:
        return ab

In [32]:
@labeling_function()
def labeling_hate(example):
    """Label tweet as propaganda if it contains the name calling tech (hate)."""
    if len(example.docs.entities) and example.hate["label"] == "offensive":
        return prop
    else:
        return ab

In [33]:
@labeling_function()
def contain_ent(example):
    """Label tweet as propaganda if it contains an entity."""
    if len(example.docs.entities):
        return prop
    else:
        return ab

In [34]:
@labeling_function()
def ent_free(example):
    """Label tweet as transparent if it doesn't contain any entities."""
    if len(example.docs.entities):
        return ab
    else:
        return gen

In [35]:
@labeling_function()
def contain_question(example):
    """Label tweet as propaganda if it contains an question."""
    for w in example.docs.sentences[0].words:
        if w.upos == "AUX":
            return prop
    return ab

In [36]:
# loading the manually crafted loaded tokens.
# extracted only from the 500 tweets used for labeling functions development.

loaded_lexicons = pd.read_csv(loaded_lex_path)["loaded-language"].to_list()
loaded_lexicons[:5]

['جاهر', 'جهر', 'تجاهر بالمعاصي', 'مجاهرة بالمعاصي', 'تجاهر بالمعاصى']

In [37]:
# loading the proppy lexicons introduced in https://arxiv.org/abs/1912.06810

proppy_lexicons = []
for file in glob.glob(proppy_lex_paths):
    with open(file, encoding="utf-8") as f:
        proppy_lexicons.extend(f.readlines())

In [38]:
# removing diacritization from proppy lexicons

pattern = r"[\u0617-\u061A\u064B-\u0652]"
proppy_lexicons = [re.sub(pattern, "", term.strip()) for term in proppy_lexicons]

In [39]:
proppy_lexicons[:5]

['إجهاض', 'إساءة', 'تعسفي', 'يقبل', 'حساب']

In [41]:
@preprocessor(memoize=False)
def tokenize_tweet(example):
    """Tokenize and lemmatize text in tweets."""
    if len(example.docs.sentences):
        example.tweet_tokens = [w.text for w in example.docs.sentences[0].words]
        example.tweet_lemmas = [w.lemma for w in example.docs.sentences[0].words]
    else:
        example.tweet_tokens = []
        example.tweet_lemmas = []
    return example

In [42]:
@preprocessor(pre=[tokenize_tweet], memoize=False)
def bigram_tweet(example):
    """Create bigrams of tweet text's tokens and lemmas."""
    example.bigram_tokens = [
        " ".join(gram) for gram in ngrams(example.tweet_tokens, 2) if len(gram)
    ]
    example.bigram_lemmas = [
        " ".join(gram) for gram in ngrams(example.tweet_lemmas, 2) if len(gram)
    ]
    return example

In [43]:
@labeling_function(pre=[bigram_tweet])
def loaded_language(example):
    """Label tweet as propaganda if it contains any of the loaded lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, loaded_lexicons)) >= 1:
        return prop
    else:
        return ab

In [44]:
@labeling_function(pre=[bigram_tweet])  # [bigram_tweet]
def genuine_language(example):
    """Label tweet as transparent if it doesn't contain any of the loaded lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, loaded_lexicons)) == 0:
        return gen
    else:
        return ab

In [ ]:
@labeling_function(pre=[bigram_tweet])
def loaded_proppy(example):
    """Label tweet as propaganda if it contains any of the proppy lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy_lexicons)) >= 1:
        return prop
    else:
        return ab

In [45]:
@labeling_function(pre=[bigram_tweet])
def genuine_proppy(example):
    """Label tweet as transparent if it doesn't contain any of the proppy lexicons."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if sum(np.in1d(tweet_ngrams, proppy_lexicons)) == 0:
        return gen
    else:
        return ab

In [46]:
@labeling_function()
def loaded_hate(example):
    """Label tweet as propaganda if it contains hate speech (loaded language)."""
    if example.hate["label"] == "offensive":
        return prop
    else:
        return ab

In [47]:
@labeling_function()
def loaded_sarcasm(example):
    """Label tweet as propaganda if it contains sarcasm (loaded language)."""
    if example.sarcasm["label"] == "Sarcasm":
        return prop
    else:
        return ab

In [48]:
@labeling_function()
def gen_hate(example):
    """Label tweet as transparent if it doesn't contain hate speech (loaded language)."""
    if example.hate["label"] == "not offensive":
        return gen
    else:
        return ab

In [49]:
@labeling_function()
def gen_sarcasm(example):
    """Label tweet as transparent if it doesn't contain sarcasm (loaded language)."""
    if example.sarcasm["label"] == "Non-Sarcasm":
        return gen
    else:
        return ab

In [50]:
# flag_waving

flag_engine = re.compile(r"Number=\w+")


@labeling_function()
def flag_wave(example):
    """Label tweet as propaganda if it contains plural pronouns (flag-waving)."""
    for w in example.docs.sentences[0].words:
        if (
            w.upos == "PRON"
            and flag_engine.findall(w.feats)[0].split("=")[-1] == "Plur"
        ):
            return prop
    return ab

In [52]:
# loading the Arabic NLTK stop words.
arabic_stop_words = stopwords.words("arabic")

# removing diacritization from stop words
pattern = r"[\u0617-\u061A\u064B-\u0652]"
arabic_stop_words = [re.sub(pattern, "", w) for w in arabic_stop_words]

In [53]:
@labeling_function(pre=[tokenize_tweet])
def repetition(example):
    """Label tweet as propaganda if it has at least one repeated token."""
    tokens = [word for word in example.tweet_tokens if word not in arabic_stop_words]
    tokens = pd.Series(tokens)
    if tokens.value_counts().max() >= 2:
        return prop
    else:
        return ab

In [54]:
# loading the WANLP propaganda dataset for distant supervision
# dataset source: https://gitlab.com/araieval/propaganda-detection


wanlp_train = pd.read_json(wanlp_prop_data_path)

label_processing = lambda x: 0 if "no technique" in x else 1
wanlp_train["labels"] = wanlp_train["labels"].apply(label_processing)

wanlp_train.head(2)

,id,text,labels
0,1358824915483435008,"#بي_بي_سي_ترندينغ: النساء ""تثرثر كثيرا"" رئيس أ...",0
1,1389927866356412416,"""ده مش معتقل ده أحسن من اللوكاندة"".. جدل وسخري...",1
2,1364082975428677632,الرجل الذي كان من فراغ https://t.co/2bnHiRqGRQ,0
3,1391667689656102912,RT @AJABreaking: عاجل | حركة حماس: ما يجري في ...,1
4,1389360446440972288,"انطلاق أسبوع المرور العربي تحت شعار: ""الحوادث ...",1


In [55]:
def clean_text(text):
    """Process text and remove links, symbols, and diacritization."""
    # links
    clean_text = re.sub(r"http\S+|t\.co/\S+", "", text)
    # mentions
    clean_text = re.sub(r"@\w+", "", clean_text)
    # hashtags
    clean_text = re.sub(r"#", "", clean_text)
    clean_text = re.sub(r"_", " ", clean_text)
    # tashqeel - from @bakriano
    clean_text = re.sub(r"[\u0617-\u061A\u064B-\u0652]", "", clean_text)
    # emojis
    clean_text = emoji.replace_emoji(clean_text, replace="")
    # remove new lines and normalize white spaces
    clean_text = re.sub(r"\s+", " ", clean_text)
    return clean_text.replace("RT :", "").strip()


wanlp_train["text"] = wanlp_train["text"].apply(clean_text)

In [56]:
encoder = fasttext.load_model(fasttext_model_path)

wanlp_hidden_states = []
for tweet in tqdm(wanlp_train.text.values, total=len(wanlp_train)):
    vec = encoder.get_sentence_vector(tweet)
    wanlp_hidden_states.append(vec)

  0%|          | 0/504 [00:00<?, ?it/s]

In [ ]:
@preprocessor()
def get_sim_scores(example):
    """Get similarity score between tweet and WANLP propaganda tweets."""
    tweet_vec = encoder.get_sentence_vector(example.text)
    sim_scores = util.cos_sim(tweet_vec, wanlp_hidden_states)
    example.sim_scores = sim_scores
    return example

In [58]:
# The included scores are fine-tuned.


@labeling_function(pre=[get_sim_scores])
def distant_supervision_prop(example):
    """Label tweet as propaganda based on its most similar WANLP example."""
    sim_scores = example.sim_scores
    most_sim = sim_scores.argmax(dim=-1).item()
    if (
        wanlp_train.labels.values[most_sim] == 1
        and sim_scores[-1][most_sim].item() >= 0.75
    ):
        return prop
    else:
        return ab

In [ ]:
# The included scores are fine-tuned.


@labeling_function(pre=[get_sim_scores])
def distant_supervision_gen(example):
    """Label tweet as transparent based on its most similar WANLP example."""
    sim_scores = example.sim_scores
    most_sim = sim_scores.argmax(dim=-1).item()
    if (
        wanlp_train.labels.values[most_sim] == 0
        and sim_scores[-1][most_sim].item() >= 0.65
    ):
        return gen
    else:
        return ab

In [59]:
@labeling_function()
def slogans(example):
    """Label tweet as propaganda if it has any of the slogans form."""
    matches = re.findall(r"لا ل\w+", example.text)
    matches += re.findall(r"نعم ل\w+", example.text)
    matches += re.findall(r"لا بديل", example.text)
    if len(matches):
        return prop
    else:
        return ab

In [60]:
# defining the manually extracted hated organizations and entities.

hitlerum = [
    "هتلر",
    "البغدادي",
    "اوردوغان",
    "قطر",
    "داعش",
    "حوثي",
    "تركيا",
    "الشيعة",
    "إيران",
    "ايران",
    "اخونجي",
    "اخوان",
    "إخوان",
    "إخوانجي",
    "أوردوغان",
    "الحوثي",
    "الحوثيين",
    "ستالين",
    "الإخوان",
    "الاخوان",
    "إرهابي",
    "الإرهابيين",
    "متطرف",
    "المتطرفين",
    "شيعي",
]

In [61]:
@labeling_function(pre=[bigram_tweet])
def reductio(example):
    """Label tweet as propaganda if it contains the Reductio Ad Hitlerum tech."""
    tweet_ngrams = example.tweet_tokens + example.tweet_lemmas
    tweet_ngrams += example.bigram_tokens + example.bigram_lemmas
    if any(np.in1d(tweet_ngrams, hitlerum)):
        return prop
    else:
        return ab

In [62]:
@labeling_function()
def exaggeration(example):
    """Label tweet as propaganda if has the "أفعل" preference form."""
    if not len(example.docs.sentences):
        return ab
    for w in example.docs.sentences[0].words:
        if w.lemma.startswith("أ") and w.upos == "ADJ":
            return prop
    return ab

In [63]:
@labeling_function()
def pronouns(example):
    """Label tweet as propaganda if has at least one pronoun."""
    pro_nouns = []
    for word in example.docs.sentences[0].words:
        if word.upos == "PRON":
            pro_nouns.append(word.text)
    if len(pro_nouns) >= 1:
        return prop
    else:
        return ab

In [64]:
# defining the XLM-RoBERTa Zero-Shot model classes
# candidate_labels = ["transparent", "propaganda"]
candidate_labels = list(unlabeled_data.xlmroberta.unique())
candidate_labels

In [65]:
# The included scores are fine-tuned.


@labeling_function()
def xlmroberta_prop(example):
    """Label tweet as propaganda based on zero-shot model."""
    if pd.isna(example.xlmroberta_label):
        return ab
    if example.xlmroberta_label["scores"][0] >= 0.90:
        if example.xlmroberta_label["labels"][0] == candidate_labels[-1]:
            return prop
        else:
            return ab
    else:
        return ab

In [ ]:
# The included scores are fine-tuned.


@labeling_function()
def xlmroberta_gen(example):
    """Label tweet as transparent based on zero-shot model."""
    if pd.isna(example.xlmroberta_label):
        return ab
    if example.xlmroberta_label["scores"][0] >= 0.90:
        if example.xlmroberta_label["labels"][0] == candidate_labels[0]:
            return gen
        else:
            return ab
    else:
        return ab

In [67]:
lfs = [
    missing_bio,
    missing_loc,
    created_2018_2019,
    created_2011_2012,
    bio_keywords,
    contain_url,
    contain_mention,
    labeling_sarcasm,
    labeling_hate,
    contain_ent,
    ent_free,
    contain_question,
    loaded_language,
    genuine_language,
    loaded_proppy,
    genuine_proppy,
    loaded_hate,
    loaded_sarcasm,
    gen_hate,
    gen_sarcasm,
    flag_wave,
    repetition,
    distant_supervision_prop,
    distant_supervision_gen,
    slogans,
    reductio,
    exaggeration,
    pronouns,
    xlmroberta_prop,
    xlmroberta_gen,
]

print(f"We have {len(lfs)} LFs used.")

In [69]:
applier = PandasLFApplier(lfs)

In [70]:
L_dev = applier.apply(labeled_data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:49<00:00, 10.11it/s]


In [72]:
results = LFAnalysis(L_dev, lfs).lf_summary(labeled_data.label.values)
results

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
bio_keywords,0,[0],0.066,0.066,0.052,32,1,0.969697
contain_url,1,[0],0.044,0.044,0.026,21,1,0.954545
labeling_irony,2,[1],0.038,0.038,0.030,7,12,0.368421
labeling_hate,3,[1],0.020,0.020,0.020,2,8,0.200000
ent_free,4,[0],0.226,0.226,0.158,108,5,0.955752
loaded_language,5,[1],0.172,0.172,0.096,32,54,0.372093
genuine_language,6,[0],0.828,0.828,0.570,398,16,0.961353
loaded_proppy,7,[1],0.636,0.636,0.560,36,282,0.113208
genuine_proppy,8,[0],0.364,0.364,0.106,170,12,0.934066
loaded_hate,9,[1],0.030,0.030,0.030,3,12,0.200000


In [73]:
L_train = applier.apply(unlabeled_data)
LFAnalysis(L_train, lfs).lf_summary()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196106/196106 [3:15:22<00:00, 16.73it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
bio_keywords,0,[0],0.325798,0.325798,0.298502
contain_url,1,[0],0.460368,0.460368,0.397515
labeling_irony,2,[1],0.039678,0.039678,0.036149
labeling_hate,3,[1],0.018306,0.018306,0.016904
ent_free,4,[0],0.198796,0.198796,0.162463
loaded_language,5,[1],0.193334,0.193334,0.157155
genuine_language,6,[0],0.806666,0.806666,0.669301
loaded_proppy,7,[1],0.771537,0.771537,0.735357
genuine_proppy,8,[0],0.228463,0.228463,0.091099
loaded_hate,9,[1],0.023309,0.023309,0.021907


In [97]:
# selecting the LFs that maximize the performance of the label model
inds = results[results["Emp. Acc."] >= 0.20].j.to_list()
inds

[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 17]

In [98]:
# displaying the names of the LFs chosen
results.iloc[inds, :].index.to_list()

['bio_keywords', 'contain_url', 'labeling_irony', 'labeling_hate', 'ent_free', 'loaded_language', 'genuine_language', 'genuine_proppy', 'loaded_hate', 'loaded_irony', 'distant_supervision_prop', 'distant_supervision_gen', 'slogans', 'reductio', 'xlmroberta_prop', 'xlmroberta_gen']


In [99]:
L_train_ = L_train[:, inds].copy()
L_dev_ = L_dev[:, inds].copy()

In [ ]:
# estimating the class balance in the unlabeled data from the labeled.

# w = labeled_data.label.value_counts(normalize=True).to_list()
# w

In [100]:
# tuning the L2 regularization parameter to maximize the performance of the label model
# we maximize the precision as it is need to surpass 50% for the noise-aware loss

best_score = 0
best_model = None
l2_values = np.arange(0.0, 0.1, 0.01)

for l2 in tqdm(l2_values, total=len(l2_values)):
    label_model = LabelModel(cardinality=2, verbose=False, device="cuda")
    label_model.fit(
        L_train=L_train_,
        n_epochs=2000,
        l2=l2,
        lr=0.01,
        seed=42,
        class_balance=None,
        progress_bar=False,
        lr_scheduler="linear",
        optimizer="adam",
        lr_scheduler_config={"warmup_percentage": 0.1, "warmup_unit": "epochs"},
    )
    score = label_model.score(L_dev_, labeled_data.label, metrics=["precision"])
    if score["precision"] >= best_score:
        best_score = score["precision"]
        best_model = label_model

  0%|          | 0/10 [00:00<?, ?it/s]

In [101]:
# displaying the configs that led to the best label model
best_model.train_config

TrainConfig(n_epochs=2000, lr=0.01, l2=0.03, optimizer='adam', optimizer_config=OptimizerConfig(sgd_config=SGDOptimizerConfig(momentum=0.9), adam_config=AdamOptimizerConfig(amsgrad=False, betas=(0.9, 0.999)), adamax_config=AdamaxOptimizerConfig(betas=(0.9, 0.999), eps=1e-08)), lr_scheduler='linear', lr_scheduler_config=LRSchedulerConfig(warmup_steps=0, warmup_unit='epochs', warmup_percentage=0.1, min_lr=0.0, exponential_config=ExponentialLRSchedulerConfig(gamma=0.9), step_config=StepLRSchedulerConfig(gamma=0.9, step_size=5)), prec_init=0.7, seed=42, log_freq=10, mu_eps=None)

Reporting the performance of the label model


In [102]:
best_model.score(
    L_dev_,
    labeled_data.label,
    tie_break_policy="abstain",
    metrics=["accuracy", "precision", "recall", "f1", "f1_macro", "f1_micro"],
)

{'accuracy': 0.858,
 'precision': 0.3707865168539326,
 'recall': 0.6875,
 'f1': 0.4817518248175182,
 'f1_macro': 0.6997403388282262,
 'f1_micro': 0.858}

In [104]:
y_true = labeled_data.label
y_pred = best_model.predict(L_dev_)
y_true = y_true[y_pred != -1]
y_pred = y_pred[y_pred != -1]

In [105]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92       452
           1       0.37      0.69      0.48        48

    accuracy                           0.86       500
   macro avg       0.67      0.78      0.70       500
weighted avg       0.91      0.86      0.88       500



Saving the weakly labeled dataset


In [184]:
labeled_data["snorkel"] = best_model.predict(L_dev_)

In [190]:
# saving the datasets
weak_preds = best_model.predict(L_train_)
unlabeled_data["label"] = weak_preds
unlabeled_data = unlabeled_data[weak_preds != -1]

In [191]:
unlabeled_data[["tweetid", "text", "label"]].to_json(weakly_labeled_data_path)